# Introduction

This guide gets you started programming in the low-level TensorFlow APIs (TensorFlow Core) using TensorFlow.jl, showing you how to:

* Manage your own TensorFlow program (a `Graph`) and TensorFlow runtime (a `Session`), instead of relying on Estimators to manage them.

* Run TensorFlow operations, using a `Session`.

* Use high level components (`datasets`, `layers`, and `feature_columns`) in this low level environment.

* Build your own training loop, instead of using the one provided by Estimators.

We recommend using the higher level APIs to build models when possible. Knowing TensorFlow Core is valuable for the following reasons:

* Experimentation and debugging are both more straight forward when you can use low level TensorFlow operations directly.

* It gives you a mental model of how things work internally when using the higher level APIs.

## Setup

Before using this guide, [install TensorFlow.jl](http://malmaud.github.io/TensorFlow.jl/latest/index.html#Installation-1).

To get the most out of this guide, you should know the following:

* How to program in Julia.

* At least a little bit about arrays.

* Ideally, something about machine learning.

In [1]:
using TensorFlow
TensorFlow.tf_version()

v"1.10.0"

## Tensor Values

The central unit of data in TensorFlow is the **tensor**. A tensor consists of a set of primitive values shaped into an array of any number of dimensions. A tensor's **rank** is its number of dimensions, while its **shape** is a vector of integers specifying the array's length along each dimension. Here are some examples of tensor values:

```julia
3. # a rank 0 tensor; a scalar with shape []

[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]

[1. 2. 3.
 4. 5. 6.] # a rank 2 tensor; a matrix with shape [2, 3]

cat([1., 7.], [2., 8.], [3., 9.]; dims=3) # a rank 3 tensor with shape [2, 1, 3]
```

TensorFlow.jl uses Julia `Array` to represent tensor **values**.

## TensorFlow Core Walkthrough

You might think of TensorFlow Core programs as consisting of two discrete sections:

1. Building the computational graph (a `Graph`).

2. Running the computational graph (using a `Session`).

### Graph

A **computational graph** is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.

* `Operation` (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.

* `Tensor`: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return `Tensor`s.

**★ Important**: `Tensor`s do not have values, they are just handles to elements in the computation graph.

Let's build a simple computational graph. The most basic operation is a constant. The Julia function that builds the operation takes a tensor value as input. The resulting operation takes no inputs. When run, it outputs the value that was passed to the constructor. We can create two floating point constants `a` and `b` as follows:

In [2]:
a = constant(3.0, dtype=Float64)
b = constant(4.0) # also Float64 implicitly; note this is different from the Python version
total = a + b
println("a = $a")
println("b = $b")
println("total = $total")

a = <Tensor Const:1 shape=() dtype=Float64>
b = <Tensor Const_2:1 shape=() dtype=Float64>
total = <Tensor Add:1 shape=() dtype=Float64>


Notice that printing the tensors does not output the values `3.0`, `4.0`, and `7.0` as you might expect. The above statements only build the computation graph. These `Tensor` objects just represent the results of the operations that will be run.

Each operation in a graph is given a unique name. This name is independent of the names the objects are assigned to in Julia. Tensors are named after the operation that produces them followed by an output index, as in "Add:1" above.

### TensorBoard

TensorFlow provides a utility called TensorBoard. One of TensorBoard's many capabilities is visualizing a computation graph. You can easily do this with a few simple commands.

First you save the computation graph to a TensorBoard summary file as follows:

In [3]:
writer = TensorFlow.summary.FileWriter(".");

This will produce an `event` file in the current directory with a name in the following format:

```
events.out.tfevents.{timestamp}.{hostname}
```

Now, launch TensorBoard with the following shell command:

In [5]:
run(`tensorboard --logdir .`)

TensorBoard 1.10.0 at http://Xingjian-Guo-Ubuntu-Desktop:6006 (Press CTRL+C to quit)


Follow the link to open TensorBoard's graphs page in your browser, and you should see a graph similar to the following:

<img src="https://www.tensorflow.org/images/getting_started_add.png" alt="TensorBoard screenshot">

Remember to interrupt the jupyter kernel to quit TensorBoard and return to Julia. Alternatively, you can run the tensorboard command in a separate terminal. For more about TensorBoard's graph visualization tools see [Visualizing learning with Tensorboard](http://malmaud.github.io/TensorFlow.jl/latest/visualization.html).

### Session

To evaluate tensors, instantiate a `Session` object, informally known as a **session**. A session encapsulates the state of the TensorFlow runtime, and runs TensorFlow operations. If a `Graph` is like a `.jl` file, a `Session` is like the `julia` executable.

The following code creates a `Session` object and then invokes its `run` method to evaluate the total tensor we created above:

In [3]:
sess = Session()
run(sess, total)

2018-10-02 13:45:07.216685: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA


7.0

When you request the output of a node with `run` TensorFlow backtracks through the graph and runs all the nodes that provide input to the requested output node. So this prints the expected value of 7.0.

You can pass multiple tensors to `run` in a vector or tuple, as in the following example:

In [4]:
run(sess, [(a,b), total])

2-element Array{Any,1}:
  [3.0, 4.0]
 7.0        

During a call to `run` any `Tensor` only has a single value. For example, the following code calls `random_uniform` to produce a `Tensor` that generates a random 3-element vector (with values in `[0,1)`):

In [5]:
vec = random_uniform([3,], dtype=Float64)
out1 = vec + 1
out2 = vec + 2
println(run(sess, vec))
println(run(sess, vec))
println(run(sess, (out1, out2)))

[0.300829, 0.87516, 0.62129]
[0.264473, 0.733504, 0.999863]
Array{Float64,1}[[1.47811, 1.8325, 1.99953], [2.47811, 2.8325, 2.99953]]


The result shows a different random value on each call to `run`, but a consistent value during a single run (`out1` and `out2` receive the same random input).

Some TensorFlow functions return `Operation`s instead of `Tensor`s. The result of calling run on an Operation is `nothing`. You run an operation to cause a side-effect, not to retrieve a value. Examples of this include the initialization, and training ops demonstrated later.

### Feeding

As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as **placeholders**. A **placeholder** is a promise to provide a value later, like a function argument.

In [6]:
x = placeholder(Float64)
y = placeholder(Float64)
z = x + y;

The preceding three lines are a bit like a function in which we define two input parameters (`x` and `y`) and then an operation on them. We can evaluate this graph with multiple inputs by using the third argument of the `run` method to feed concrete values to the placeholders:

In [12]:
println(run(sess, z, Dict(x => 3, y => 4.5)))
println(run(sess, z, Dict(x => [1, 3], y => [2, 4])))

7.5
[3.0, 7.0]


Also note that the third argument can be used to overwrite any tensor in the graph. The only difference between placeholders and other `Tensor`s is that placeholders throw an error if no value is fed to them.

## Datasets

WIP (PyCall?)

## Layers

WIP (PyCall?)

## Feature columns

WIP (PyCall?)

## Training

Now that you're familiar with the basics of core TensorFlow, let's train a small regression model manually.

### Define the data

First let's define some inputs, `x`, and the expected output for each input, `y_true`:

In [20]:
x = constant([1., 2., 3., 4.])
y_true = constant([0., -1., -2., -3.]);

### Define the model

WIP

## Next steps

To learn more about building models with TensorFlow consider the following:

* Custom Estimators, to learn how to build customized models with TensorFlow. Your knowledge of TensorFlow Core will help you understand and debug your own models. [WIP]

If you want to learn more about the inner workings of TensorFlow consider the following documents, which go into more depth on many of the topics discussed here:

* Graphs and Sessions [WIP]
* Tensors [WIP]
* Variables [WIP]

----------------------
*Except as otherwise noted, the content of this page is licensed under the [Creative Commons Attribution 3.0 License](https://creativecommons.org/licenses/by/3.0/), and code samples are licensed under the [Apache 2.0 License](https://www.apache.org/licenses/LICENSE-2.0). This notebook is adapted from the official [TensorFlow Guide on Low Level APIs](https://www.tensorflow.org/guide/low_level_intro).*